<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Создание-выборок" data-toc-modified-id="Создание-выборок-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Создание выборок</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Light-GBM" data-toc-modified-id="Light-GBM-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Light GBM</a></span></li><li><span><a href="#Тестовая-выборка" data-toc-modified-id="Тестовая-выборка-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Тестовая выборка</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 



## Подготовка

Подготовим и загрузим нужные библиотеки.

In [1]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle


import lightgbm as lgb

Откроем файл, убедимся в правильности представления данных.

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv',index_col=[0])
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Названия столбцов ожидаемые, предполагаем, что столбец 'text' в дальнейшем будет признаком, а столбец 'toxic' - целевым признаком. Удостоверимся, что в данных нет пропусков и дубликатов.

In [3]:
#количество пропусков
df.isna().sum()

text     0
toxic    0
dtype: int64

In [4]:
#количество дубликатов
df.duplicated().sum()

0

Какое количество токсичных комментариев встречается в датасете?

In [5]:
df['toxic'].mean()

0.10161213369158527

Около 10% комментариев - токсичны.

Так как работа с целым датафреймом у меня занимает очень много времени, всё компилируется около 4 часов, поэтому я сокращаю датасет до 100 000 строк.

In [6]:
df['toxic'].sum()

16186

## Обучение

Прежде чем переходить к обучению моделей, очистим текст от лишнего и создадим обучающие и тестовые выборки.

### Создание выборок

Напишем функцию для очистки текста с помощью регулярных выражений.

In [7]:
def clear_text(text):
    text = re.sub(r'[^a-zA-z]', ' ', text)
    return " ".join(text.split()).lower()

Лемматизируем текст с помощью функций.

In [8]:
#import nltk
nltk.download('punkt')
nltk.download('omw-1.4')

#подгружаем дополнительную библиотеку слов 
nltk.download('wordnet')

#подгружаем тэги
nltk.download('averaged_perceptron_tagger')


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    
#создание словаря тэгов
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def lem(text):      
    lemmatizer = WordNetLemmatizer()

#полученные слова объединяем в один элемент столбца 
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    return lemmatized_output

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
#проверка на простом примере
sentence = "The striped bats are hanging on their feet for best"
lem(sentence)

'The strip bat be hang on their foot for best'

Применяем описанные функции к столбцу 'text'

In [10]:
#лемматизация
df['text'] = df['text'].apply(lem)
df.head()

,text,toxic
0,Explanation Why the edits make under my userna...,0
1,D'aww ! He match this background colour I 'm s...,0
2,"Hey man , I 'm really not try to edit war . It...",0
3,`` More I ca n't make any real suggestion on i...,0
4,"You , sir , be my hero . Any chance you rememb...",0


Очищаем текст

In [11]:
df['text'] = df['text'].apply(clear_text)
df.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,`` more i ca n t make any real suggestion on i...,0
4,you sir be my hero any chance you remember wha...,0


Разбиваем df на признаки и целевой признак.

In [12]:
#определяем признаки
x = df.drop(['toxic'],axis=1) 

#определяем целевой признак
y = df['toxic'] 

Разобьем признаки на обучающую и валидационную выборку в соотношении 3:1.

In [13]:
#разбиваем на выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify = y, random_state=12345)

#проверка размера выборок
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((119469, 1), (39823, 1), (119469,), (39823,))

Выведем количество токсичных комментариев выборки y_train и ее размер для понимания соотношения.

In [14]:
y_train.sum(), x_train.shape

(12140, (119469, 1))

Создадим корпуса для признаков из обучающей и тестовой выборки.

In [15]:
#создание корпуса для обучающей выборки
corpus_train = x_train['text'].values

#создание корпуса для тестовой выборки
corpus_test = x_test['text'].values

Подключим библиотеку стоп-слов

In [16]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Убираем частые и неинформативные слова. Метрику tf_idf принимаем за признаки. Векторизатор должеен быть обучен только на тренировочной части данных. 

In [17]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords,sublinear_tf=True)

#признаки обучающей выборки 
tf_idf_train = count_tf_idf.fit_transform(corpus_train)

#признаки тестовой выборки
tf_idf_test = count_tf_idf.transform(corpus_test)
"Размер матрицы:", tf_idf_train.shape

('Размер матрицы:', (119469, 138685))

### Логистическая регрессия

Переходим к обучению моделей. Первой будет Логистическая регрессия. 

In [18]:
model_lr = LogisticRegression(random_state=12345, class_weight = 'balanced', C=9, solver='sag')

Обучаем логистичекую регрессию, для подбора гиперпараметров засечем время.

In [19]:
%%time
model_lr.fit(tf_idf_train,y_train)

CPU times: user 13.8 s, sys: 38.1 ms, total: 13.9 s
Wall time: 13.9 s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=9, class_weight='balanced', random_state=12345,
                   solver='sag')

Создаем предсказания. Модели лучше оценивать кросс-валидацией, она даёт более стабильную оценку. На тренировочной выборке модели часто переобучены.

In [20]:
mean_score = cross_val_score(model_lr, tf_idf_train, y_train, scoring="f1", cv = 3).mean()
mean_score

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.7626732679181881

### Light GBM

Проверим, как с этой задачей справится LGBM

In [21]:
clf = lgb.LGBMClassifier(class_weight='balanced',max_bin=5)
clf.fit(tf_idf_train, y_train)

LGBMClassifier(class_weight='balanced', max_bin=5)

Находим метрику f1 на обучающей выборке с помощью кросс-валидации

In [22]:
mean_score = cross_val_score(clf, tf_idf_train, y_train, scoring="f1", cv = 3).mean()
mean_score

0.7281192644797768

### Тестовая выборка

Модели обучены, теперь предстоит проверить их на тестовой выборке. Метрика f1 была больше на модели логистической регресси, поэтому проводим тест на ней.

In [23]:
#логистическая регрессия
predict_test = model_lr.predict(tf_idf_test)
f1_score(y_test, predict_test)

0.7647804540739888

## Выводы

Целью проекта было обучить модель распределять комментарии на нейтральные и токсичные. В представленном датасете было около 10% токсичных комментариев. Для решения поставленной задачи сначала текст был лемматизирован и очищен с учётом стоп-слов, в нем остались только символы латинского алфавита. 
Обучающая и тестовая выборка были составлены в соотношении 3:1. 
Были использованы такие модели как логистическая регрессия и Light GBM. 
Обе модели показали хороший результат метрики f1, но логистическая регрессия на тестовой выборке была лучше и показала  значение 0.76